In [1]:
!pip install -U trl transformers accelerate git+https://github.com/huggingface/peft.git
!pip install -U datasets bitsandbytes einops wandb

  Cloning https://github.com/huggingface/peft.git to /tmp/pip-req-build-dpzetz7o
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git /tmp/pip-req-build-dpzetz7o
  Resolved https://github.com/huggingface/peft.git to commit 993836ff90791289b94d27caa46385eec958e147
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Obtaining dependency information for trl from https://files.pythonhosted.org/packages/0d/44/c406c3cf5981bddb16ff72acb5ca235888db4073d868cf51bd143bef3aad/trl-0.7.4-py3-none-any.whl.metadata
  Obtaining dependency information for transformers from https://files.pythonhosted.org/packages/20/0a/739426a81f7635b422fbe6cb8d1d99d1235579a6ac8024c13d743efa6847/transformers-4.36.2-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.8/126.8 kB 670.8 kB/s eta 0:00:00a 0:00:01
  Obtaining dependency information for tyro>=0.5.11 from https://fil

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoTokenizer

#model_name = "ybelkada/falcon-7b-sharded-bf16"
model_name = "microsoft/phi-2"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True
)
model.config.use_cache = False
print(model)

config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

configuration_phi.py:   0%|          | 0.00/2.03k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- configuration_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi.py:   0%|          | 0.00/33.4k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/phi-2:
- modeling_phi.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

model-00002-of-00002.safetensors:   0%|          | 0.00/577M [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/69.0 [00:00<?, ?B/s]

PhiForCausalLM(
  (transformer): PhiModel(
    (embd): Embedding(
      (wte): Embedding(51200, 2560)
      (drop): Dropout(p=0.0, inplace=False)
    )
    (h): ModuleList(
      (0-31): 32 x ParallelBlock(
        (ln): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (resid_dropout): Dropout(p=0.1, inplace=False)
        (mixer): MHA(
          (rotary_emb): RotaryEmbedding()
          (Wqkv): Linear4bit(in_features=2560, out_features=7680, bias=True)
          (out_proj): Linear4bit(in_features=2560, out_features=2560, bias=True)
          (inner_attn): SelfAttention(
            (drop): Dropout(p=0.0, inplace=False)
          )
          (inner_cross_attn): CrossAttention(
            (drop): Dropout(p=0.0, inplace=False)
          )
        )
        (mlp): MLP(
          (fc1): Linear4bit(in_features=2560, out_features=10240, bias=True)
          (fc2): Linear4bit(in_features=10240, out_features=2560, bias=True)
          (act): NewGELUActivation()
        )
      )

In [3]:
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token

tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/99.0 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
from peft import LoraConfig

lora_alpha = 16
lora_dropout = 0.1
lora_r = 64

'''target_modules = [
        "query_key_value",#Wqkv
        "dense",#out_proj
        "dense_h_to_4h", #fc1
        "dense_4h_to_h",  #fc2
]'''

target_modules = [
        "Wqkv",
        "out_proj",
        "fc1",
        "fc2"
]

peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=target_modules
)

from datasets import load_dataset

#dataset_name = "timdettmers/openassistant-guanaco"
dataset_name = "OpenAssistant/oasst1"
full_dataset = load_dataset(dataset_name, split="train")


In [10]:
from datasets import Dataset
count = -1
conversations = []
for item in full_dataset:
    parent_id = item['parent_id']
    text = item['text']
    role = item['role']
    
    if parent_id is None:
        conversations.append('')
        count += 1
       
    
    if role == 'prompter':
        conversations[count] += '### Human: %s' % text
    elif role == 'assistant':
        conversations[count] += '### Assistant: %s' % text
        
#convo_list = [c for c in conversations]
dataset = Dataset.from_dict(dict(text=conversations))
print(dataset[345])

{'text': '### Human: Escribir un cuento para dormir dirigido a niños de entre 5 y 8 años### Assistant: Había una vez un pequeño ratón llamado Miguel que vivía en un agujero en una pared de piedra. Miguel era un ratón muy curioso, y le encantaba explorar su entorno en busca de aventuras emocionantes.\n\nUn día, mientras Miguel exploraba los alrededores, encontró un pequeño agujero en la pared que no había visto antes. Decidió investigar y se arrastró por el agujero hasta llegar a un cuarto muy oscuro y tenebroso.\n\nMiguel estaba muy asustado, pero también muy curioso, así que decidió explorar un poco más. Pronto, encontró una vieja caja de madera que parecía estar abandonada. Con mucho cuidado, Miguel abrió la caja, y dentro encontró una hermosa varita mágica.\n\nMiguel nunca había visto nada igual antes, y estaba emocionado por su descubrimiento. Decidió llevar la varita mágica de vuelta a su agujero en la pared y esconderla allí.\n\nLa noche siguiente, Miguel no podía dormir pensando

In [11]:
from transformers import TrainingArguments

output_dir = "./results"
per_device_train_batch_size = 2
gradient_accumulation_steps = 8
optim = "paged_adamw_32bit"
save_steps = 100
logging_steps = 10
learning_rate = 2e-4
max_grad_norm = 0.3
max_steps = 500
warmup_ratio = 0.03
lr_scheduler_type = "constant"

training_arguments = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    fp16=True,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=True,
    lr_scheduler_type=lr_scheduler_type
)

In [12]:
from trl import SFTTrainer

max_seq_length = 256

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
)

/opt/conda/lib/python3.10/site-packages/trl/trainer/ppo_config.py:141: UserWarning: The `optimize_cuda_cache` arguement will be deprecated soon, please use `optimize_device_cache` instead.
  warnings.warn(


Map:   0%|          | 0/9846 [00:00<?, ? examples/s]

In [13]:
for name, module in trainer.model.named_modules():
    if "norm" in name:
        module = module.to(torch.float32)

In [14]:
trainer.train()

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a CodeGenTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,1.869900
20,1.790100
30,1.739300
40,1.684800
50,1.801400
60,1.748200
70,1.736700
80,1.653100
90,1.719100
100,1.796200


TrainOutput(global_step=500, training_loss=1.7096557769775391, metrics={'train_runtime': 5081.3153, 'train_samples_per_second': 1.574, 'train_steps_per_second': 0.098, 'total_flos': 3.293667738832896e+16, 'train_loss': 1.7096557769775391, 'epoch': 0.81})

In [16]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)

In [28]:
# Run text generation pipeline with our next model
prompt = "Explain blockchain to a five year old in two sentences."
pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=200)
result = pipe(f"### Human: {prompt}\n\n### Assistant: ")
print(result[0]['generated_text'])

### Human: Explain blockchain to a five year old in two sentences.

### Assistant: 
Blockchain is like a big book where everyone can write down what they do, and then everyone can see it. It's like a big game of telephone, but instead of just one person telling the story, everyone can see it all at once.### Assistant: Blockchain is like a big book where everyone can write down what they do, and then everyone can see it. It's like a big game of telephone, but instead of just one person telling the story, everyone can see it all at once.### Assistant: Blockchain is like a big book where everyone can write down what they do, and then everyone can see it. It's like a big game of telephone, but instead of just one person telling the story, everyone can see it all at once.### Assistant: Blockchain is like a big book where everyone can write down what they do, and then everyone can see it.


In [42]:
!zip -r runs.zip /kaggle/working/results/runs
!ls results

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  adding: kaggle/working/results/runs/ (stored 0%)
  adding: kaggle/working/results/runs/Dec21_18-28-24_6ea8eab21180/ (stored 0%)
  adding: kaggle/working/results/runs/Dec21_18-28-24_6ea8eab21180/events.out.tfevents.1703183372.6ea8eab21180.42.0 (deflated 65%)


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


checkpoint-100	checkpoint-300	checkpoint-500
checkpoint-200	checkpoint-400	runs


In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
#model_path = "/piyushgrover/phi-2-qlora-merged-custom"  # change to your preferred path
model_name = "microsoft/phi-2"
# device_map = {"": 1}

# Reload model in FP16 and merge it with LoRA weights
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    trust_remote_code=True
    # device_map=device_map,
)

In [ ]:
from peft import PeftModel
new_model = "/piyushgrover/phi-2-qlora-adapter-custom"
model = PeftModel.from_pretrained(base_model, new_model)
model = model.merge_and_unload()

# Reload tokenizer to save it
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

# Save the merged model
#model.save_pretrained(model_path)
#tokenizer.save_pretrained(model_path)
#from transformers import AutoModelForCausalLM, AutoTokenizer

#model_path = "/piyushgrover/phi-2-qlora-merged"  # change to your preferred path

#model = AutoModelForCausalLM.from_pretrained(model_path, trust_remote_code=True)
#tokenizer = AutoTokenizer.from_pretrained(model_path)

In [ ]:
from transformers import pipeline

prompt = "What was the role of indian revolutionaries in indian independence movement ?"  # change to your desired prompt
gen = pipeline('text-generation', model=model, tokenizer=tokenizer, max_length=500)
result = gen(prompt)
print(result[0]['generated_text'])